# Motivation

The motivation for creating prediction and ranking tasks for EIE datasets was to extend the dataset ecosystem available for algorithmic fairness research in two directions: the education domain and the dataset from Eastern Europe (Ukraine). We obtained the data from publicly available Open Data resource https://zno.testportal.com.ua/opendata.


The tasks were created by Dr. Julia Stoyanovich and Andrew Bell from the Center for Responsible AI, New York University, and Tetiana Zakharchenko, Nazarii Drushchak, Oleksandra Konopatska, and Olha Liuba from Ukrainian Catholic University.


The creation of the dataset was funded by the Center for Responsible AI, New York University.

# Composition of Dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

We will load and work with datasets from 2016 to 2021 since they have a comparable structure. We downloadedoriginal datasets from Open Data Resource and uploaded to HuggingFace for easiness of usage.

## Loading

Load datasets from hugging face

In [2]:
# load datasets
datasets = {}
years = range(2016, 2023)
for year in years:
    print(f"EIE {year} Loading...")
    if int(year) >= 2019:
        file_name = f'Odata{year}File.csv'
    else:
        file_name = f'OpenData{year}.csv'
    try:           
        dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')
    except:
        dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')
    datasets.update({year:dataset})
    print("success")

EIE 2016 Loading...
success
EIE 2017 Loading...
success
EIE 2018 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_37537/3686568105.py:11: DtypeWarning: Columns (119) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')


success
EIE 2019 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_37537/3686568105.py:13: DtypeWarning: Columns (100,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


success
EIE 2020 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_37537/3686568105.py:13: DtypeWarning: Columns (96,97,98,100,102,103,104,105,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


success
EIE 2021 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_37537/3686568105.py:11: DtypeWarning: Columns (117,118,119,121,123,124,125,126,137,138,139,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')


success
EIE 2022 Loading...
success


Names of all features are the same across all years, but can have some difference with capital and lower cases. That's why we lower all of them. Also, we add new columns – a corresponding year and age for the further research. 

In [3]:
for year, dataset in datasets.items():
    #lowercase all columns and add year as attribute
    dataset.columns = [col.lower() for col in dataset.columns]
    #add year before merging datasets
    dataset['year'] = year
    #add age at the moment of registration for the exam before merging datasets
    dataset['age'] = year - dataset['birth']

In [4]:
for year, dataset in datasets.items():
    print(year, dataset.shape)

2016 (268003, 108)
2017 (240889, 121)
2018 (335687, 127)
2019 (353813, 128)
2020 (379299, 128)
2021 (389323, 149)
2022 (234104, 31)


## Unifiying column names

Find datasets that have the maximum common list of columns.

In [5]:
def find_dataframes_with_most_common_columns(dataframes):
    max_common_columns = 0
    dataframes_with_max_common_columns = []

    # Find all combinations of dataframes
    combinations = list(itertools.combinations(dataframes, 3))

    for df1, df2, df3 in combinations:
        common_columns = list(set(df1.columns) & set(df2.columns) & set(df3.columns))
        num_common_columns = len(common_columns)

        if num_common_columns > max_common_columns:
            max_common_columns = num_common_columns
            dataframes_with_max_common_columns = [(df1, df2, df3)]
        elif num_common_columns == max_common_columns:
            dataframes_with_max_common_columns.append((df1, df2, df3))

    return max_common_columns, dataframes_with_max_common_columns


In [6]:
result = find_dataframes_with_most_common_columns(datasets.values())
print(f'{result[0]} common columns')
print(f"Dataframes with the most common columns: {result[1][0][0].year.unique()}, {result[1][0][1].year.unique()}, {result[1][0][2].year.unique()}")

128 common columns
Dataframes with the most common columns: [2019], [2020], [2021]


For 2019 and 2020 we have exactly the same list of columns, which will be our base

In [7]:
set(datasets[2019].columns) ^ set(datasets[2020].columns)

set()

### Check for 2021 and 2020

In [8]:
set(datasets[2021].columns) - set(datasets[2020].columns)

{'mathdpalevel',
 'mathstball',
 'mathstball12',
 'mathstlang',
 'mathstptareaname',
 'mathstptname',
 'mathstptregname',
 'mathstpttername',
 'mathsttest',
 'mathstteststatus',
 'ukrsubtest',
 'umladaptscale',
 'umlball',
 'umlball100',
 'umlball12',
 'umlptareaname',
 'umlptname',
 'umlptregname',
 'umlpttername',
 'umltest',
 'umlteststatus'}

It was a new experiment in 2021 with standard level of Mathematics and ukrsubtest, that failed since it was cancelled due to Covid.

In [9]:
datasets[2021].drop(columns=[col for col in datasets[2021].columns if col.startswith('mathst')], inplace = True)

In [10]:
datasets[2021].drop(columns=['ukrsubtest'], inplace = True)

In [11]:
set(datasets[2021].columns) - set(datasets[2020].columns)

{'mathdpalevel',
 'umladaptscale',
 'umlball',
 'umlball100',
 'umlball12',
 'umlptareaname',
 'umlptname',
 'umlptregname',
 'umlpttername',
 'umltest',
 'umlteststatus'}

Before 2021 there was UKRtest, that corresponds to Ukrainian language and literature, and it was obligatory for all participants who wants to pursue the high level education. 

In 2021 it was decided to create separate test for Ukrainian language only, make it obligatory and leave the previous version with literature as a possible choise. Thus, Ukrtest was renamed to UMLtest (for Ukrainian language and Ukrainian literature), and the new test with Ukrainian language only got name UKRtest, that was before. 

To avoid confusion with the subjects in merged dataframe we rename 'uml' by 'ukr', and 'ukr' for 2021 rename as 'ukrlang', since it is a new subject.


In [12]:
datasets[2021].rename(columns=lambda x: 'ukrlang' + x[3:] if x.startswith('ukr') else x, inplace = True)
datasets[2021].rename(columns=lambda x: 'ukr' + x[3:] if x.startswith('uml') else x, inplace = True)

In [13]:
set(datasets[2021].columns) - set(datasets[2020].columns)

{'mathdpalevel',
 'ukrlangadaptscale',
 'ukrlangball',
 'ukrlangball100',
 'ukrlangball12',
 'ukrlangptareaname',
 'ukrlangptname',
 'ukrlangptregname',
 'ukrlangpttername',
 'ukrlangtest',
 'ukrlangteststatus'}

### 2016 and 2019

compare 2016 and 2019 structure

In [14]:
set(datasets[2016].columns)-set(datasets[2019].columns)

{'frball100',
 'frptareaname',
 'frptname',
 'frptregname',
 'frpttername',
 'frtest',
 'frteststatus',
 'rusball100',
 'rusptareaname',
 'rusptname',
 'rusptregname',
 'ruspttername',
 'rustest',
 'rusteststatus',
 'spball100',
 'spptareaname',
 'spptname',
 'spptregname',
 'sppttername',
 'sptest',
 'spteststatus'}

Fix the issue with 'fr' and 'fra' for French language test and 'sp' and 'spa' with Spanish language test

In [15]:
datasets[2016].columns = [col[:2]+'a'+col[2:] if col.startswith('fr') or col.startswith('sp') else col for col in datasets[2016].columns]

In [16]:
set(datasets[2016].columns)-set(datasets[2019].columns)

{'rusball100',
 'rusptareaname',
 'rusptname',
 'rusptregname',
 'ruspttername',
 'rustest',
 'rusteststatus'}

In 2016 and 2017 Ukraine had Russian test, but from 2018 it was cancelled.

### Compare 2017 and 2019

In [17]:
set(datasets[2017].columns)-set(datasets[2019].columns)

{'rusball100',
 'rusball12',
 'rusptareaname',
 'rusptname',
 'rusptregname',
 'ruspttername',
 'rustest',
 'rusteststatus',
 'stid'}

In [18]:
datasets[2017].head(5)

,outid,stid,birth,sextypename,regname,areaname,tername,regtypename,tertypename,classprofilename,...,rustest,rusteststatus,rusball100,rusball12,rusptname,rusptregname,rusptareaname,ruspttername,year,age
0,EDDDC0A3-C615-4101-96B3-9F1A4C654A2C,14809269-9F28-454C-8B3B-01DE067C91F3,1999,жіноча,Житомирська область,Новоград-Волинський район,смт Городниця,Випускник минулих років,місто,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,18
1,3528FBEB-5ABE-4956-B715-68FABAE0A566,C5B85614-D95B-4029-B3E4-920246DA7B17,1999,чоловіча,Сумська область,Сумська область,м.Лебедин,"Учень (слухач, студент) професійно-технічного,...",місто,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,18
2,1BA2DD60-B125-4DB0-875A-9AB8E54B4411,E1BB0166-3FFE-4BA0-BFB7-D60EC8E8CD8E,1991,чоловіча,Миколаївська область,м.Миколаїв,Корабельний район міста,Випускник минулих років,місто,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,26
3,29BD1575-16B2-4EAD-B2AB-DC2A3365DE1E,574350E1-DDA9-47A8-ADAA-6F8915BD2A05,2000,чоловіча,Донецька область,Донецька область,м.Покровськ,Випускник загальноосвітнього навчального закла...,місто,Фізико-математичний,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,17
4,FF8BFD9C-837B-4F38-A321-C9AC264F83E6,4E01CF4C-3596-4178-A44D-556197D73E7B,2000,чоловіча,Івано-Франківська область,Івано-Франківська область,м.Івано-Франківськ,"Учень (слухач, студент) професійно-технічного,...",місто,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,17


Drop stid from 2017

In [19]:
datasets[2017].drop(columns = 'stid', inplace=True)

### Compare 2018 and 2019

In [20]:
set(datasets[2018].columns)-set(datasets[2019].columns)

set()

### compare 2019 and 2022

In 2022 because of the war it was provided different test. It was computerbased, tested only 3 subjects, the level of tasks were as a half of the previous level, and it was provided in one day. Everyone should be veru careful with interpretation of thed data for 2022

In [21]:
set(datasets[2022].columns)-set(datasets[2019].columns)

{'block1',
 'block1ball',
 'block1ball100',
 'block2',
 'block2ball',
 'block2ball100',
 'block3',
 'block3ball',
 'block3ball100',
 'ptareaname',
 'ptregname',
 'pttername',
 'test',
 'testdate',
 'teststatus'}

block1, block2, block3 are names of the corresponding subjects

In [22]:
print(datasets[2022].block1.unique())
print(datasets[2022].block2.unique())
print(datasets[2022].block3.unique())

['Українська мова']
['Історія України']
['Математика']


So, block1 tested Ukrainian language, block2 History of Ukraine, and block3 Mathematics.

In [23]:
datasets[2021].ukrlangtest.unique()

array(['Українська мова', nan], dtype=object)

In [24]:
datasets[2021].mathtest.unique()

array(['Математика', nan], dtype=object)

In [25]:
datasets[2021].histtest.unique()

array(['Історія України', nan], dtype=object)

In [26]:
#rename columns
datasets[2022].rename(columns = {'block1':'ukrlangtest', 'block2':'mathtest', 'block3': 'histtest'},inplace = True)
datasets[2022].rename(columns=lambda x: 'ukrlang' + x[6:] if x.startswith('block1') else x, inplace = True)
datasets[2022].rename(columns=lambda x: 'hist' + x[6:] if x.startswith('block2') else x, inplace = True)
datasets[2022].rename(columns=lambda x: 'math' + x[6:] if x.startswith('block3') else x, inplace = True)

In [27]:
set(datasets[2022].columns)-set(datasets[2019].columns)

{'ptareaname',
 'ptregname',
 'pttername',
 'test',
 'testdate',
 'teststatus',
 'ukrlangball',
 'ukrlangball100',
 'ukrlangtest'}

Before 2022, each subject test was provided in different days and location. In 2022 it was one test for 3 subjects simultaneously, so the location for all of them is equal.

In [28]:
subj = ['ukrlang', 'hist', 'math']

for sub in subj:
    datasets[2022][sub+'ptareaname'] = datasets[2022]['ptareaname']
    datasets[2022][sub+'ptregname'] = datasets[2022]['ptregname']
    datasets[2022][sub+'pttername'] = datasets[2022]['pttername']
    datasets[2022][sub+'teststatus'] = datasets[2022]['teststatus']


In [29]:
datasets[2022].drop(columns = ['ptareaname', 'ptregname','pttername','teststatus'] , inplace = True)

In [30]:
set(datasets[2022].columns)-set(datasets[2021].columns)

{'test', 'testdate'}

A 'test' feature is used to name the new test.

In [31]:
datasets[2022].test.unique()

array(['НМТ '], dtype=object)

In [32]:
datasets[2022].test.replace('НМТ ', 'NMT', inplace = True)

In [ ]:
dataset[2022].rename(columns={'test':'test_type'}, inplace=True)

There is no such a feature for previous years, so let's create it.

In [33]:
for year in datasets:
    if year<2022:
        datasets[year]['test_type'] = 'EIE'

In [34]:
datasets[2016]

,outid,birth,sextypename,regname,areaname,tername,regtypename,eoname,eotypename,eoregname,...,rustest,rusteststatus,rusball100,rusptname,rusptregname,rusptareaname,ruspttername,year,age,test_type
0,a99c6c63-aa70-4aec-ba42-370f7261e857,1998,чоловіча,Запорізька область,Мелітопольський район,с.Терпіння,Випускник загальноосвітнього навчального закла...,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",колегіум,Запорізька область,...,NaN,Не зареєстровано для проходження основної сесії,NaN,NaN,NaN,NaN,NaN,2016,18,EIE
1,c3136421-569e-422e-ae8f-41c4c931fd70,1998,жіноча,Хмельницька область,Красилівський район,м.Красилів,Випускник загальноосвітнього навчального закла...,Красилівська загальноосвітня школа I-III ступе...,середня загальноосвітня школа,Хмельницька область,...,NaN,Не зареєстровано для проходження основної сесії,NaN,NaN,NaN,NaN,NaN,2016,18,EIE
2,fbd0e280-4b77-4f34-942d-00bc850c2992,1995,жіноча,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,Випускник минулих років,NaN,NaN,NaN,...,NaN,Не зареєстровано для проходження основної сесії,NaN,NaN,NaN,NaN,NaN,2016,21,EIE
3,30de395e-7a74-452a-8370-6856d240fbfb,1999,чоловіча,Чернівецька область,м.Чернівці,Шевченківський район міста,Випускник загальноосвітнього навчального закла...,Чернівецька спеціалізована школа І-ІІІ ступені...,спеціалізована школа,Чернівецька область,...,NaN,Не зареєстровано для проходження основної сесії,NaN,NaN,NaN,NaN,NaN,2016,17,EIE
4,52a40c74-374e-4744-bd6f-938ab7a4a641,1997,чоловіча,Миколаївська область,Врадіївський район,с.Кумарі,"Учень (слухач, студент) професійно-технічного,...",NaN,NaN,NaN,...,NaN,Не зареєстровано для проходження основної сесії,NaN,NaN,NaN,NaN,NaN,2016,19,EIE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267998,7ac30c16-091a-4d22-9a1c-ec868b4c2c66,1998,жіноча,Дніпропетровська область,Дніпропетровська область,м.Орджонікідзе,Випускник загальноосвітнього навчального закла...,"Комунальний заклад ""Середня загальноосвітня шк...",середня загальноосвітня школа,Дніпропетровська область,...,NaN,Не зареєстровано для проходження основної сесії,NaN,NaN,NaN,NaN,NaN,2016,18,EIE
267999,6437b74b-50bf-43f9-a5e7-a5118b4b003e,1998,жіноча,Дніпропетровська область,Нікопольський район,с.Капулівка (Покровська сільрада),Випускник минулих років,NaN,NaN,NaN,...,NaN,Не зареєстровано для проходження основної сесії,NaN,NaN,NaN,NaN,NaN,2016,18,EIE
268000,0e6bf6cd-fd2c-41f6-bae2-b867e3e65095,1999,жіноча,Хмельницька область,Ярмолинецький район,с.Томашівка,Випускник загальноосвітнього навчального закла...,Ярмолинецький технологічний ліцей,ліцей,Хмельницька область,...,NaN,Не зареєстровано для проходження основної сесії,NaN,NaN,NaN,NaN,NaN,2016,17,EIE
268001,7c94eec3-4f4b-4f46-abde-da1a6a0d3ddc,1998,жіноча,Луганська область,Старобільський район,с.Чмирівка,Випускник загальноосвітнього навчального закла...,Чмирівський навчально-виховний комплекс «Школа...,навчально-виховний комплекс,Луганська область,...,NaN,Не зареєстровано для проходження основної сесії,NaN,NaN,NaN,NaN,NaN,2016,18,EIE


The only one new feature that we have is a testdate.

In [35]:
set(datasets[2022].columns)-set(datasets[2021].columns)

{'test', 'testdate'}

## Merging datasets

In [36]:
# concat datasets
df_all = pd.concat(datasets.values())

In [37]:
df_all.shape

(2201118, 150)

In [38]:
list(df_all.columns)

['outid',
 'birth',
 'sextypename',
 'regname',
 'areaname',
 'tername',
 'regtypename',
 'eoname',
 'eotypename',
 'eoregname',
 'eoareaname',
 'eotername',
 'eoparent',
 'ukrtest',
 'ukrteststatus',
 'ukrball100',
 'ukrball12',
 'ukrptname',
 'ukrptregname',
 'ukrptareaname',
 'ukrpttername',
 'histtest',
 'histlang',
 'histteststatus',
 'histball100',
 'histball12',
 'histptname',
 'histptregname',
 'histptareaname',
 'histpttername',
 'mathtest',
 'mathlang',
 'mathteststatus',
 'mathball100',
 'mathball12',
 'mathptname',
 'mathptregname',
 'mathptareaname',
 'mathpttername',
 'phystest',
 'physlang',
 'physteststatus',
 'physball100',
 'physptname',
 'physptregname',
 'physptareaname',
 'physpttername',
 'chemtest',
 'chemlang',
 'chemteststatus',
 'chemball100',
 'chemptname',
 'chemptregname',
 'chemptareaname',
 'chempttername',
 'biotest',
 'biolang',
 'bioteststatus',
 'bioball100',
 'bioptname',
 'bioptregname',
 'bioptareaname',
 'biopttername',
 'geotest',
 'geolang',
 'g

Save datasets to csv format.

In [39]:
df_all.to_csv('eie_all.csv',index=False)

Having created a dataset once, we can load it immediately in the future

In [ ]:
df_all = pd.read_csv('eie_all.csv')

## Exploring dataset and unifying values of some columns

In [ ]:
df_all.head()

In [ ]:
df_all[df_all.regtypename.isna()]

In [ ]:
df_all.info()

In [ ]:
dict(df_all.dtypes)

In Ukraine we use ',' to write a decimals, so sometimes it lead to confusion. Let's fix it

In [ ]:
for col in df_all.columns:
    if '100' in col:
        df_all[col] = pd.to_numeric(df_all[col].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))

In [ ]:
dict(df_all.dtypes)

In [ ]:
df_all['ukrteststatus'].unique()

In 2016 and 2019 we have different phrases for the same status.

In [ ]:
for col in df_all.columns:
    if 'teststatus' in col:
        df_all[col] = df_all[col].str.replace('’', "'")
        df_all[col] = df_all[col].replace('Не зареєстровано для проходження основної сесії', np.nan)
        df_all[col] = df_all[col].replace('Не склав', 'Не подолав поріг')
        df_all[col] = df_all[col].replace('Отримав результат', 'Зараховано')

In [ ]:
df_all['ukrteststatus'].unique()

## Unifying 'tertypename' feature

Since in 2021 'tertypename' has 3 territory types: city (місто), town (селище міського типу) and village (селище, село), and before we had only 2: city (місто) and village (селище, село), we reconstruct the correct type for 2018-2020 period.

In [ ]:
def encode_tertypename(x):
    if x['tername'].startswith('м.'):
        return 'місто'
    elif x['tername'].startswith('смт'):
        return 'селище міського типу'
    elif x['tername'].startswith('с.'):
        return 'селище, село'
    elif x['tername'].startswith('с-ще'):
         return 'селище, село'
    elif x['areaname'].startswith('м.'):
        return 'місто'
    return None
        
df_all['tertypename'] = df_all.apply(encode_tertypename,axis=1)

In [ ]:
df_all.tertypename.unique()

In [ ]:
df_all[df_all.tertypename.isna()]

In [ ]:
df_all.drop(df_all.loc[df_all['outid']=='3b07c9d4-067c-47fa-8cf4-8e2097ade009'].index, inplace=True)

In [ ]:
df_all.shape

In [ ]:
df_all.tail()

## Unifying 'regtypename' feature

In [ ]:
df_all.regtypename.unique()

In [ ]:
regtypename_dict = {
    'Випускник української школи поточного року': ['Випускник загальноосвітнього навчального закладу 2016 року',
                                                    'Випускник загальноосвітнього навчального закладу 2017 року',
                                                    'Випускник закладу загальної середньої освіти 2018 року',
                                                    'Випускник закладу загальної середньої освіти 2019 року',
                                                    'випускник закладу загальної середньої освіти 2020 року',
                                                    'Випускник загальноосвітнього навчального закладу 2021 року',
                                                    'Випускник закладу загальної середньої освіти 2022 року'],
    'Випускник іноземної школи': ['Випускник, який здобуде в 2016 році повну загальну середню освіту в навчальному закладі іншої держави', 
                                    'Випускник, який здобуде в 2017 році повну загальну середню освіту в навчальному закладі іншої держави',
                                    'Випускник, який здобуде в 2018 році повну загальну середню освіту в навчальному закладі іншої держави', 
                                    'Випускник, який здобуде в 2019 році повну загальну середню освіту в закордонному закладі освіти',
                                    'Випускник, який здобуде в 2020 році повну загальну середню освіту в закордонному закладі освіти', 
                                    'Випускник, який здобуде в 2021 році повну загальну середню освіту в навчальному закладі іншої держави',
                                    'Випускник, який здобуде в 2022 році повну загальну середню освіту в закордонному закладі освіти'],
    'Учень коледжу закладу професійної (професійно-технічної) або вищої освіти': ['Учень (слухач) закладу професійної (професійно-технічної) освіти', 
                                                                                    'Учень (слухач, студент) професійно-технічного, вищого навчального закладу', 
                                                                                    'Студент закладу вищої/фахової передвищої освіти',
                                                                                    'Студент закладу вищої освіти'],
    'Випускник минулих років': ['Випускник минулих років'],
    'Установа виконання покарань':['Установа виконання покарань']
}


In [ ]:
df_all[df_all.year == 2017].regtypename.unique()

In [ ]:
regtypename_detailed_dict = {
    'Випускник української школи поточного року': ['Випускник загальноосвітнього навчального закладу 2016 року',
                                                    'Випускник загальноосвітнього навчального закладу 2017 року',
                                                    'Випускник закладу загальної середньої освіти 2018 року',
                                                    'Випускник закладу загальної середньої освіти 2019 року',
                                                    'випускник закладу загальної середньої освіти 2020 року',
                                                    'Випускник загальноосвітнього навчального закладу 2021 року',
                                                    'Випускник закладу загальної середньої освіти 2022 року'],
    'Випускник іноземної школи': ['Випускник, який здобуде в 2016 році повну загальну середню освіту в навчальному закладі іншої держави', 
                                    'Випускник, який здобуде в 2017 році повну загальну середню освіту в навчальному закладі іншої держави',
                                    'Випускник, який здобуде в 2018 році повну загальну середню освіту в навчальному закладі іншої держави', 
                                    'Випускник, який здобуде в 2019 році повну загальну середню освіту в закордонному закладі освіти',
                                    'Випускник, який здобуде в 2020 році повну загальну середню освіту в закордонному закладі освіти', 
                                    'Випускник, який здобуде в 2021 році повну загальну середню освіту в навчальному закладі іншої держави',
                                    'Випускник, який здобуде в 2022 році повну загальну середню освіту в закордонному закладі освіти'],
    None : ['Учень (слухач, студент) професійно-технічного, вищого навчального закладу'],
    'Студент закладу вищої освіти': ['Студент закладу вищої освіти', 
                                      'Студент закладу вищої/фахової передвищої освіти'],
    'Учень (слухач) закладу професійної (професійно-технічної) освіти':['Учень (слухач) закладу професійної (професійно-технічної) освіти'],
    'Випускник минулих років': ['Випускник минулих років'],
    'Установа виконання покарань':['Установа виконання покарань']
}

In [ ]:
def encode_regtypename_detailed(x):
    for type, possible_types in regtypename_detailed_dict.items():
        if x in possible_types:
            return type
    return None

def encode_regtypename(x):
    for type, possible_types in regtypename_dict.items():
        if x in possible_types:
            return type
    return None

df_all['regtypename_detailed'] = df_all.regtypename.map(encode_regtypename_detailed)
df_all.regtypename = df_all.regtypename.map(encode_regtypename)

In [ ]:
df_all.regtypename.unique()

In [ ]:
df_all['regtypename_detailed'].unique()

In [ ]:
list(df_all.columns)

In [ ]:
df_all.shape

In [ ]:
df_all.to_csv('eie_preprocessed.csv', index=False)